In [1]:
print("hello")

hello


## Q & A over the code bsea to understand how its works

#### first it will extract all the code from the link and create a knowledge base based in the code space if what we 
#### have done and gives the explanation of the code

In [1]:
import os
from git import Repo   # for this we take the link (GitPython) to clone the repository from the github
from langchain.text_splitter import Language   # language detection
from langchain_community.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
# create a folder
!mkdir test_repo

In [3]:
# in this test_repo folder we are gonna clone the repository

repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/kuldeep1909/Ent-to-End-Medical-Chatbot-Genai", to_path=repo_path)


In [4]:
# load all the code from the repo

loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser=LanguageParser(language= Language.PYTHON, parser_threshold=500))

In [6]:
documents = loader.load()

In [7]:
documents

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, request, jsonify\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_groq import ChatGroq\nfrom langchain.vectorstores import Pinecone\nimport pinecone\n\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import  ChatPromptTemplate\nfrom dotenv import load_dotenv\n\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nPINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")\nGROQ_API_KEY = os.environ.get("GROQ_API_KEY")\n\nos.environ[\'PINECONE_API_KEY\'] = PINECONE_API_KEY\nos.environ[\'GROQ_API_KEY\'] = GROQ_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n#####################################################################\nindex_name = "medibot"\n\ndocsearch = Pinecone.from_existing_inde

In [8]:
len(documents)

7

In [9]:
documents[0]

Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, request, jsonify\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_groq import ChatGroq\nfrom langchain.vectorstores import Pinecone\nimport pinecone\n\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import  ChatPromptTemplate\nfrom dotenv import load_dotenv\n\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nPINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")\nGROQ_API_KEY = os.environ.get("GROQ_API_KEY")\n\nos.environ[\'PINECONE_API_KEY\'] = PINECONE_API_KEY\nos.environ[\'GROQ_API_KEY\'] = GROQ_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n#####################################################################\nindex_name = "medibot"\n\ndocsearch = Pinecone.from_existing_index

In [10]:
# context aware splittig of the code

documents_splitter = RecursiveCharacterTextSplitter.from_language(language= Language.PYTHON,
                                                                  chunk_size= 500,
                                                                  chunk_overlap= 20)

In [11]:
texts = documents_splitter.split_documents(documents)

In [12]:
texts

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, request, jsonify\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_groq import ChatGroq\nfrom langchain.vectorstores import Pinecone\nimport pinecone\n\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import  ChatPromptTemplate\nfrom dotenv import load_dotenv\n\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)'),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")\nGROQ_API_KEY = os.environ.get("GROQ_API_KEY")\n\nos.environ[\'PINECONE_API_KEY\'] = PINECONE_API_KEY\nos.environ[\'GROQ_API_KEY\'] = GROQ_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n#############################

In [13]:
len(texts)

15

In [15]:
!pip -q install sentence-transformers

In [16]:
# we need to conver the code into embeddings

from langchain.embeddings import  HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="microsoft/codebert-base")

c:\Users\malvi\anaconda3\envs\sourcecode\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.
c:\Users\malvi\anaconda3\envs\sourcecode\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\malvi\.cache\huggingface\hub\models--microsoft--codebert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symli

In [18]:
test = embeddings.embed_query("Hello Kuldeep")
print(len(test))

768


In [20]:
!pip -q install chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.14 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.
langchain-core 0.3.29 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.19 which is incompatible.
langchain-groq 0.1.10 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.29 which is incompatible.
pydantic-settings 2.7.1 requires pydantic>=2.7.0, but you have pydantic 1.10.19 which is incompatible.


In [25]:

from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory= './db')

In [26]:
# to activate this vectorstore
vectorstore.persist()

C:\Users\malvi\AppData\Local\Temp\ipykernel_16552\1303152908.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [27]:
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = GROQ_API_KEY


In [32]:
!pip install "pydantic<2.0.0" --force-reinstall

  Using cached pydantic-1.10.19-cp310-cp310-win_amd64.whl.metadata (153 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pydantic-1.10.19-cp310-cp310-win_amd64.whl (2.1 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.4
    Uninstalling pydantic-2.10.4:
      Successfully uninstalled pydantic-2.10.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.14 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.
langchain-core 0.3.29 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.19 which is incompatible.
langchain-groq 0.1.10 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.29 which is incompatible.
pydantic-settings 2.7.1 requires pydantic>=2.7.0, but you have pydantic 1.10.19 which is incompatible.


In [33]:
!pip install --force-reinstall langchain-groq

  Using cached langchain_groq-0.2.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached groq-0.13.1-py3-none-any.whl.metadata (14 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [34]:
## initialise the llm from groq


from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key = GROQ_API_KEY,
    model_name = "llama-3.3-70b-specdec")


In [35]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

C:\Users\malvi\AppData\Local\Temp\ipykernel_16552\1764516168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [42]:
## Create the final chian
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [43]:
question = "what is download_hugging_face_embeddings function"

In [44]:
qa(question)

Number of requested results 20 is greater than number of elements in index 15, updating n_results = 15


{'question': 'what is download_hugging_face_embeddings function',
 'chat_history': [SystemMessage(content='', additional_kwargs={}, response_metadata={})],
 'answer': 'The `download_hugging_face_embeddings` function is used to download and initialize a Hugging Face embeddings model. \n\nIn this specific code, it is defined as:\n\n```python\ndef download_hugging_face_embeddings():\n    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")\n    return embeddings\n```\n\nThis function uses the `HuggingFaceEmbeddings` class from the `langchain.embeddings` module to create an instance of the "sentence-transformers/all-MiniLM-L6-v2" model. This model is a pre-trained language model that can be used for various natural language processing tasks, such as text classification, sentiment analysis, and more.\n\nThe `download_hugging_face_embeddings` function returns an instance of the `HuggingFaceEmbeddings` class, which can be used to generate embeddings for inpu

In [47]:
question = "what is load_pdf_data function"
print(qa(question))

Number of requested results 20 is greater than number of elements in index 15, updating n_results = 15


{'question': 'what is load_pdf_data function', 'chat_history': [SystemMessage(content='The human asks about the `download_hugging_face_embeddings` function. The `download_hugging_face_embeddings` function is used to download and initialize a Hugging Face embeddings model, specifically the "sentence-transformers/all-MiniLM-L6-v2" model, which is a pre-trained language model for various natural language processing tasks. The function returns an instance of the `HuggingFaceEmbeddings` class, used to generate embeddings for input text, and in this context, is applied to generate embeddings from text extracted from PDF files for creating a Pinecone index. The human also inquires about the RecursiveCharacterTextSplitter function, which is a text splitting utility from the Langchain library that splits documents into smaller chunks based on a specified character limit, and in the provided code, it is used to split the extracted PDF data into chunks of 1000 characters with a 20-character overl